# Домашнее задание

## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [59]:
import pandas as pd

In [60]:
log = pd.read_csv('visit_log.csv', sep=';')

In [61]:
log.loc[log['traffic_source'].isin(['yandex', 'google']), 'source_type'] = 'organic'
log.loc[(log['traffic_source'] == 'paid') & (log['region'] == 'Russia'), 'source_type'] = 'ad'
log.loc[(log['traffic_source'] == 'paid') & (log['region'] != 'Russia'), 'source_type'] = 'other'
log.loc[pd.isnull(log['source_type']), 'source_type'] = log['traffic_source']

In [62]:
log.sample(15)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
5705,1550015037,786600e78e,https://host.ru/f1eb4601740d627ab0,Russia,fb784a3235,google,organic
6476,1550019587,de5b9681f0,https://host.ru/8fe7c39e5c598bbc54,Russia,eb1992c911,direct,direct
16945,1550082106,b169ee467d,https://host.ru/a8ce0b447c10a1deaa,Russia,018a672f5b,direct,direct
11322,1550048358,f9446c9ab7,https://host.ru/05e41944dd85a900a6,Russia,75694966fc,direct,direct
5589,1550014347,9e0f5beee4,https://host.ru/060d9bdbcba66b8bb3,Ukraine,f408063d5f,google,organic
11907,1550051886,c6acbee305,https://host.ru/8fe7c39e5c598bbc54,Russia,8bdc89bd4e,google,organic
18702,1550092938,4e8f68d06b,https://host.ru/1afc5618562365020a,Russia,1599b8256b,yandex,organic
11023,1550046505,b0c2893a5b,https://host.ru/108ce4b365afb7b88e,Russia,b9662350f8,direct,direct
8934,1550034118,40647bbbac,https://host.ru/b8b58337d272ee7b15,Russia,44bf62780c,direct,direct
16062,1550076699,dd99357441,https://host.ru/da83c74e35e7806fe0,Belarus,9d7d3f46fd,email,email


## Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [63]:
df = pd.read_csv('URLs.txt')
df = df[df['url'].str.contains('\/[\d]{8}-')]

In [66]:
df.head(10)

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
10,/middleeast/36131117-divizion-s-400-ne-zametil...
11,/economics/36065674-rossiyane-vozmutilis-minim...
12,/politics/36118047-otets-24-detey-potreboval-i...
13,/travel/36194479-v-sheremetevo-passazhiry-ustr...
14,/politics/35638742-nesolidno-mariya-zaharova-s...


## Задание 3

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [69]:
df = pd.read_csv('ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [70]:
# Подсчитано количество оценок одного пользователя
df_sorted = df.groupby('userId').count()['rating'].reset_index().sort_values('rating', ascending=False)
# Создаем датафрейм, где остаются пользователи с количеством оценок больше 100
df_sorted = df_sorted.loc[df_sorted.rating > 100]
df_sorted = df_sorted.drop(['rating'], axis='columns')
joined = df_sorted.merge(df, how='left', on='userId')

In [71]:
# Функцию для подсчета разницы между максимальным и минимальным значением timestamp
def counting(data):
    return data.timestamp.max() - data.timestamp.min()

In [72]:
mean_time = obj.apply(counting).mean()

In [73]:
print(f"Среднее время жизни составляет {mean_time/86400} дней")

Среднее время жизни составляет 463.8947621482917 дней


## Задание 4

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). 
<br /> Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

<br /> Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [74]:
#Что, где, когда?